In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.optimize import minimize
import pyreadr
import os
import time
from scipy.stats import norm
import copy
import warnings
import utils, regression_fit
import pickle
from importlib import reload
reload(utils)
reload(regression_fit)
from utils import *
from regression_fit import *

home = os.getcwd()

## Dataset

In [3]:
GroupA = pyreadr.read_r(home + '\\Other_data\\GroupA.rda')
GroupA_df = pd.DataFrame(list(GroupA.values())[0])

GroupB = pyreadr.read_r(home + '\\Other_data\\GroupB.rda')
GroupB_df = pd.DataFrame(list(GroupB.values())[0])

GroupC = pyreadr.read_r(home + '\\Other_data\\GroupC.rda')
GroupC_df = pd.DataFrame(list(GroupC.values())[0])

GroupD = pyreadr.read_r(home + '\\Other_data\\GroupD.rda')
GroupD_df = pd.DataFrame(list(GroupD.values())[0])

GroupE = pyreadr.read_r(home + '\\Other_data\\GroupE.rda')
GroupE_df = pd.DataFrame(list(GroupE.values())[0])

GroupF = pyreadr.read_r(home + '\\Other_data\\GroupF.rda')
GroupF_df = pd.DataFrame(list(GroupF.values())[0])

GroupG = pyreadr.read_r(home + '\\Other_data\\GroupG.rda')
GroupG_df = pd.DataFrame(list(GroupG.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupK = pyreadr.read_r(home + '\\Other_data\\GroupK.rda')
GroupK_df = pd.DataFrame(list(GroupK.values())[0])

GroupL = pyreadr.read_r(home + '\\Other_data\\GroupL.rda')
GroupL_df = pd.DataFrame(list(GroupL.values())[0])

GroupM = pyreadr.read_r(home + '\\Other_data\\GroupM.rda')
GroupM_df = pd.DataFrame(list(GroupM.values())[0])

GroupN = pyreadr.read_r(home + '\\Other_data\\GroupN.rda')
GroupN_df = pd.DataFrame(list(GroupN.values())[0])

GroupP = pyreadr.read_r(home + '\\Other_data\\GroupP.rda')
GroupP_df = pd.DataFrame(list(GroupP.values())[0])

In [4]:
regions_df = [GroupA_df, GroupB_df, GroupC_df, GroupD_df, GroupE_df, GroupF_df, GroupG_df, GroupH_df, GroupJ_df, GroupK_df,
             GroupL_df, GroupM_df, GroupN_df, GroupP_df]

idx_2020 = 17516
idx_2021 = 35104

In [5]:
# Adding columns y0_96, y0_336, diff_96, diff_336 to the dataset of each region
for df in regions_df:   
    df = add_autoregressive_features(df)

regions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P']
train_set = {}
test_set = {}

# The dataset of each region is split in train and test set
for (reg, df) in zip(regions, regions_df):
    # Train set
    train_set[reg] = df[720:87440] 
    #Test set
    test_set[reg] = df[87440:]

### The goal is to reproduce the results of paper [1] on all regions

[1] Joseph de Vilmarest, Jethro Browell, Matteo Fasiolo, Yannig Goude and Olivier Wintenberger, Adaptive Probabilistic Forecasting of Electricity
(Net-)Load

# Persistence benchmark (2 days)

The prediction for each target time is equal to the value of 48h before


In [35]:
y_pers_benchmark = {}

for reg, df in zip(regions, regions_df):
    pb_reg = Persistence_Benchmark(df)
    y_pers_benchmark[reg] = pb_reg.predict(test_set[reg]['targetTime'])

In [36]:
y_mat = np.zeros((len(test_set['A']),len(regions)))
for i,reg in enumerate(regions):
    y_mat[:,i] = test_set[reg]['node']
    
yhat_persistence = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_persistence[:,i] = y_pers_benchmark[reg]

# GAM models obtained with the mgcv package in R

### Offline GAM
Results obtained with the mgcv package in R by using the bam function

In [7]:
Y_GAM_R_bam = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupA_GAM_R_bam.rda')
y_GAM_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_GAM_R_bam["A"] = np.array(y_GAM_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupB_GAM_R_bam.rda')
y_GAM_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_GAM_R_bam["B"] = np.array(y_GAM_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupC_GAM_R_bam.rda')
y_GAM_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_GAM_R_bam["C"] = np.array(y_GAM_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupD_GAM_R_bam.rda')
y_GAM_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_GAM_R_bam["D"] = np.array(y_GAM_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupE_GAM_R_bam.rda')
y_GAM_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_GAM_R_bam["E"] = np.array(y_GAM_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupF_GAM_R_bam.rda')
y_GAM_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_GAM_R_bam["F"] = np.array(y_GAM_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupG_GAM_R_bam.rda')
y_GAM_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_GAM_R_bam["G"] = np.array(y_GAM_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_GAM_R_bam.rda')
y_GAM_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_GAM_R_bam["H"] = np.array(y_GAM_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_GAM_R_bam.rda')
y_GAM_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_GAM_R_bam["J"] = np.array(y_GAM_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_GAM_R_bam.rda')
y_GAM_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_GAM_R_bam["H"] = np.array(y_GAM_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_GAM_R_bam.rda')
y_GAM_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_GAM_R_bam["J"] = np.array(y_GAM_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupK_GAM_R_bam.rda')
y_GAM_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_GAM_R_bam["K"] = np.array(y_GAM_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupL_GAM_R_bam.rda')
y_GAM_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_GAM_R_bam["L"] = np.array(y_GAM_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupM_GAM_R_bam.rda')
y_GAM_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_GAM_R_bam["M"] = np.array(y_GAM_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupN_GAM_R_bam.rda')
y_GAM_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_GAM_R_bam["N"] = np.array(y_GAM_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupP_GAM_R_bam.rda')
y_GAM_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_GAM_R_bam["P"] = np.array(y_GAM_R_P).flatten()

In [8]:
idx_2020 = 17516
idx_2021 = 35104

yhat_GAM_mat_R_bam = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_GAM_mat_R_bam[:,i] = Y_GAM_R_bam[reg]

print("\nResults from R (Offline GAM)")

print("\nnRMSE GAM on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_GAM_mat_R_bam[:idx_2020]))
print("nRMSE GAM on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_GAM_mat_R_bam[idx_2020:idx_2021]))
print("nRMSE GAM on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_GAM_mat_R_bam[idx_2021:]))

print("\nnMAE GAM on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_GAM_mat_R_bam[:idx_2020]))
print("nMAE GAM on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_GAM_mat_R_bam[idx_2020:idx_2021]))
print("nMAE GAM on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_GAM_mat_R_bam[idx_2021:]))



Results from R (Offline GAM)

nRMSE GAM on 2019 test set:  0.3554482982247092
nRMSE GAM on 2020 test set:  0.48519379380764477
nRMSE GAM on 2021 test set:  0.6351325513177465

nMAE GAM on 2019 test set:  0.32637142975291267
nMAE GAM on 2020 test set:  0.4524598866590619
nMAE GAM on 2021 test set:  0.6011450814429554


# GAM with yearly data update

In [9]:
Y_bam_upd_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupA_bam_yearly_updated.rda')
Y_bam_upd_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_bam_upd_R["A"] = np.array(Y_bam_upd_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupB_bam_yearly_updated.rda')
Y_bam_upd_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_bam_upd_R["B"] = np.array(Y_bam_upd_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupC_bam_yearly_updated.rda')
Y_bam_upd_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_bam_upd_R["C"] = np.array(Y_bam_upd_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupD_bam_yearly_updated.rda')
Y_bam_upd_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_bam_upd_R["D"] = np.array(Y_bam_upd_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupE_bam_yearly_updated.rda')
Y_bam_upd_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_bam_upd_R["E"] = np.array(Y_bam_upd_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupF_bam_yearly_updated.rda')
Y_bam_upd_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_bam_upd_R["F"] = np.array(Y_bam_upd_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupG_bam_yearly_updated.rda')
Y_bam_upd_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_bam_upd_R["G"] = np.array(Y_bam_upd_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_bam_yearly_updated.rda')
Y_bam_upd_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_bam_upd_R["H"] = np.array(Y_bam_upd_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_bam_yearly_updated.rda')
Y_bam_upd_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_bam_upd_R["J"] = np.array(Y_bam_upd_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_bam_yearly_updated.rda')
Y_bam_upd_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_bam_upd_R["H"] = np.array(Y_bam_upd_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_bam_yearly_updated.rda')
Y_bam_upd_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_bam_upd_R["J"] = np.array(Y_bam_upd_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupK_bam_yearly_updated.rda')
Y_bam_upd_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_bam_upd_R["K"] = np.array(Y_bam_upd_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupL_bam_yearly_updated.rda')
Y_bam_upd_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_bam_upd_R["L"] = np.array(Y_bam_upd_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupM_bam_yearly_updated.rda')
Y_bam_upd_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_bam_upd_R["M"] = np.array(Y_bam_upd_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupN_bam_yearly_updated.rda')
Y_bam_upd_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_bam_upd_R["N"] = np.array(Y_bam_upd_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupP_bam_yearly_updated.rda')
Y_bam_upd_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_bam_upd_R["P"] = np.array(Y_bam_upd_R_P).flatten()

In [10]:
idx_2020 = 17516
idx_2021 = 35104

yhat_bam_upd_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_bam_upd_mat_R[:,i] = Y_bam_upd_R[reg]

print("\nResults from R (GAM with yearly update)")

print("\nnRMSE GAM on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_bam_upd_mat_R[:idx_2020]))
print("nRMSE GAM on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_bam_upd_mat_R[idx_2020:idx_2021]))
print("nRMSE GAM on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_bam_upd_mat_R[idx_2021:]))

print("\nnMAE GAM on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_bam_upd_mat_R[:idx_2020]))
print("nMAE GAM on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_bam_upd_mat_R[idx_2020:idx_2021]))
print("nMAE GAM on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_bam_upd_mat_R[idx_2021:]))



Results from R (GAM with yearly update)

nRMSE GAM on 2019 test set:  0.3554482982247092
nRMSE GAM on 2020 test set:  0.4068986898024555
nRMSE GAM on 2021 test set:  0.38727066486845235

nMAE GAM on 2019 test set:  0.32637142975291267
nMAE GAM on 2020 test set:  0.37582938984397096
nMAE GAM on 2021 test set:  0.3777251247822882


# Static Kalman Filter trained in R (with the obtained offline GAM model)

### 48h delay considered due to data availability

In [20]:
Y_KF_static_delay_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupA_KF_static_delay.Rda')
Y_7d_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_KF_static_delay_R["A"] = np.array(Y_7d_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupB_KF_static_delay.Rda')
Y_7d_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_KF_static_delay_R["B"] = np.array(Y_7d_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupC_KF_static_delay.Rda')
Y_7d_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_KF_static_delay_R["C"] = np.array(Y_7d_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupD_KF_static_delay.Rda')
Y_7d_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_KF_static_delay_R["D"] = np.array(Y_7d_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupE_KF_static_delay.Rda')
Y_7d_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_KF_static_delay_R["E"] = np.array(Y_7d_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupF_KF_static_delay.Rda')
Y_7d_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_KF_static_delay_R["F"] = np.array(Y_7d_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupG_KF_static_delay.Rda')
Y_7d_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_KF_static_delay_R["G"] = np.array(Y_7d_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_static_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_static_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_static_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_static_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_static_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_static_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_static_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_static_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupK_KF_static_delay.Rda')
Y_7d_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_KF_static_delay_R["K"] = np.array(Y_7d_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupL_KF_static_delay.Rda')
Y_7d_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_KF_static_delay_R["L"] = np.array(Y_7d_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupM_KF_static_delay.Rda')
Y_7d_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_KF_static_delay_R["M"] = np.array(Y_7d_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupN_KF_static_delay.Rda')
Y_7d_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_KF_static_delay_R["N"] = np.array(Y_7d_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupP_KF_static_delay.Rda')
Y_7d_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_KF_static_delay_R["P"] = np.array(Y_7d_R_P).flatten()

In [21]:
idx_2020 = 17516
idx_2021 = 35104

yhat_KF_static_delay_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_KF_static_delay_mat_R[:,i] = Y_KF_static_delay_R[reg]
    
print("\nResults from R - Static KF")

print("\nnRMSE Static KF with 48h delay on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_KF_static_delay_mat_R[:idx_2020]))
print("nRMSE Static KF with 48h delay on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_KF_static_delay_mat_R[idx_2020:idx_2021]))
print("nRMSE Static KF with 48h delay on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_KF_static_delay_mat_R[idx_2021:]))

print("\nnMAE Static KF with 48h delay on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_KF_static_delay_mat_R[:idx_2020]))
print("nMAE Static KF with 48h delay on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_KF_static_delay_mat_R[idx_2020:idx_2021]))
print("nMAE Static KF with 48h delay on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_KF_static_delay_mat_R[idx_2021:]))



Results from R - Static KF

nRMSE Static KF with 48h delay on 2019 test set:  0.338203943863475
nRMSE Static KF with 48h delay on 2020 test set:  0.3755293802401946
nRMSE Static KF with 48h delay on 2021 test set:  0.3822969574009974

nMAE Static KF with 48h delay on 2019 test set:  0.3078849363929946
nMAE Static KF with 48h delay on 2020 test set:  0.3491162527377908
nMAE Static KF with 48h delay on 2021 test set:  0.3705900767346633


# Dynamic Kalman Filter trained in R (with the obtained offline GAM model)

### 48h delay considered due to data availability

Variances of the Kalman Filter are obtained by maximizing the likelihood on the train set.

The matrices obtained with viking in R produce results far from the ones of the paper.

In [28]:
Y_KF_dynamic_delay_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupA_KF_dynamic_delay.Rda')
Y_7d_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_KF_dynamic_delay_R["A"] = np.array(Y_7d_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupB_KF_dynamic_delay.Rda')
Y_7d_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_KF_dynamic_delay_R["B"] = np.array(Y_7d_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupC_KF_dynamic_delay.Rda')
Y_7d_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_KF_dynamic_delay_R["C"] = np.array(Y_7d_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupD_KF_dynamic_delay.Rda')
Y_7d_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_KF_dynamic_delay_R["D"] = np.array(Y_7d_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupE_KF_dynamic_delay.Rda')
Y_7d_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_KF_dynamic_delay_R["E"] = np.array(Y_7d_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupF_KF_dynamic_delay.Rda')
Y_7d_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_KF_dynamic_delay_R["F"] = np.array(Y_7d_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupG_KF_dynamic_delay.Rda')
Y_7d_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_KF_dynamic_delay_R["G"] = np.array(Y_7d_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupK_KF_dynamic_delay.Rda')
Y_7d_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_KF_dynamic_delay_R["K"] = np.array(Y_7d_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupL_KF_dynamic_delay.Rda')
Y_7d_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_KF_dynamic_delay_R["L"] = np.array(Y_7d_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupM_KF_dynamic_delay.Rda')
Y_7d_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_KF_dynamic_delay_R["M"] = np.array(Y_7d_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupN_KF_dynamic_delay.Rda')
Y_7d_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_KF_dynamic_delay_R["N"] = np.array(Y_7d_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupP_KF_dynamic_delay.Rda')
Y_7d_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_KF_dynamic_delay_R["P"] = np.array(Y_7d_R_P).flatten()

In [29]:
yhat_KF_dynamic_delay_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_KF_dynamic_delay_mat_R[:,i] = Y_KF_dynamic_delay_R[reg]

In [30]:
idx_2020 = 17516
idx_2021 = 35104

print("\nResults from R")

print("\nnRMSE dynamic KF on bam with 48h delay on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nRMSE dynamic KF on bam with 48h delay on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nRMSE dynamic KF on bam with 48h delay on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))

print("\nnMAE dynamic KF on bam with 48h delay on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nMAE dynamic KF on bam with 48h delay on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nMAE dynamic KF on bam with 48h delay on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))



Results from R

nRMSE dynamic KF on bam with 48h delay on 2019 test set:  0.329952977787414
nRMSE dynamic KF on bam with 48h delay on 2020 test set:  0.3455705472500563
nRMSE dynamic KF on bam with 48h delay on 2021 test set:  0.34534577444377396

nMAE dynamic KF on bam with 48h delay on 2019 test set:  0.29795220579731285
nMAE dynamic KF on bam with 48h delay on 2020 test set:  0.3199031132443549
nMAE dynamic KF on bam with 48h delay on 2021 test set:  0.32521421080013396


Dynamic Kalman Filter results by using variances sigma=0.08 and Q=1e-13*I_(d+1).

In [37]:
Y_KF_dynamic_delay_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupA_KF_dynamic_delay_fxd.Rda')
Y_7d_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_KF_dynamic_delay_R["A"] = np.array(Y_7d_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupB_KF_dynamic_delay_fxd.Rda')
Y_7d_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_KF_dynamic_delay_R["B"] = np.array(Y_7d_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupC_KF_dynamic_delay_fxd.Rda')
Y_7d_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_KF_dynamic_delay_R["C"] = np.array(Y_7d_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupD_KF_dynamic_delay_fxd.Rda')
Y_7d_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_KF_dynamic_delay_R["D"] = np.array(Y_7d_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupE_KF_dynamic_delay_fxd.Rda')
Y_7d_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_KF_dynamic_delay_R["E"] = np.array(Y_7d_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupF_KF_dynamic_delay_fxd.Rda')
Y_7d_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_KF_dynamic_delay_R["F"] = np.array(Y_7d_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupG_KF_dynamic_delay_fxd.Rda')
Y_7d_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_KF_dynamic_delay_R["G"] = np.array(Y_7d_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay_fxd.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay_fxd.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay_fxd.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay_fxd.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupK_KF_dynamic_delay_fxd.Rda')
Y_7d_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_KF_dynamic_delay_R["K"] = np.array(Y_7d_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupL_KF_dynamic_delay_fxd.Rda')
Y_7d_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_KF_dynamic_delay_R["L"] = np.array(Y_7d_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupM_KF_dynamic_delay_fxd.Rda')
Y_7d_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_KF_dynamic_delay_R["M"] = np.array(Y_7d_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupN_KF_dynamic_delay_fxd.Rda')
Y_7d_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_KF_dynamic_delay_R["N"] = np.array(Y_7d_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupP_KF_dynamic_delay_fxd.Rda')
Y_7d_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_KF_dynamic_delay_R["P"] = np.array(Y_7d_R_P).flatten()

In [38]:
yhat_KF_dynamic_delay_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_KF_dynamic_delay_mat_R[:,i] = Y_KF_dynamic_delay_R[reg]

In [39]:
idx_2020 = 17516
idx_2021 = 35104

print("\nResults from R")

print("\nnRMSE dynamic KF on bam with 48h delay on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nRMSE dynamic KF on bam with 48h delay on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nRMSE dynamic KF on bam with 48h delay on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))

print("\nnMAE dynamic KF on bam with 48h delay on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nMAE dynamic KF on bam with 48h delay on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nMAE dynamic KF on bam with 48h delay on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))



Results from R

nRMSE dynamic KF on bam with 48h delay on 2019 test set:  0.3297880584496303
nRMSE dynamic KF on bam with 48h delay on 2020 test set:  0.3581101792040419
nRMSE dynamic KF on bam with 48h delay on 2021 test set:  0.3326826859509367

nMAE dynamic KF on bam with 48h delay on 2019 test set:  0.2979100742046042
nMAE dynamic KF on bam with 48h delay on 2020 test set:  0.302373242152572
nMAE dynamic KF on bam with 48h delay on 2021 test set:  0.30774732384628717


# Kalman Filter on all regions in Python

The main problem of this approach is that the GAM model in python obtains results which are worse than the ones we rely upon in R

In [16]:
features = ['t', 't2', 'doy_s', 'doy_c', 'doy_s2', 'doy_c2', 'node_sm', 'clock_hour', 'dow_RpH', 'School_Hol', 
           'x2T_weighted.mean_p_max_point', 'x2Tsm_point', 'SSRD_mean_2_Cap', 'WindSpd100_weighted.mean_cell','EMBEDDED_WIND_CAPACITY',
           'WindSpd10_weighted.mean_cell', 'TP_weighted.mean_cell', 'y0_336', 'y0_96']
features_to_standardize = ['node_sm','clock_hour','x2T_weighted.mean_p_max_point', 'x2Tsm_point', 'SSRD_mean_2_Cap', 'WindSpd100_weighted.mean_cell','EMBEDDED_WIND_CAPACITY',
                          'WindSpd10_weighted.mean_cell', 'TP_weighted.mean_cell']


def dataset_preprocessing(df, features, features_to_standardize, exog_name):
    df_std = standardize_dataframe(df[features_to_standardize])
    for f in list(set(features) - set(features_to_standardize+['t2'])):
        df_std[f] = df[f]
    df_std['t2'] = df_std['t']**2
    df_std['dow_RpH'] = 1*(df_std['dow_RpH']=='Lun')+2*(df_std['dow_RpH']=='Mar')+\
                            3*(df_std['dow_RpH']=='Mer')+4*(df_std['dow_RpH']=='Jeu')+\
                            5*(df_std['dow_RpH']=='Ven')+6*(df_std['dow_RpH']=='Sam')+\
                            7*(df_std['dow_RpH']=='Dim')+8*(df_std['dow_RpH']=='Autumn Break')+\
                            9*(df_std['dow_RpH']=='Christmas Holiday')+10*(df_std['dow_RpH']=='Easter Holidays')+\
                            11*(df_std['dow_RpH']=='Summer Break')+12*(df_std['dow_RpH']=='Summer Holiday')+\
                            13*(df_std['dow_RpH']=='February Half Term')
    df_std['School_Hol'] = 1*(df_std['School_Hol']=='N')+2*(df_std['School_Hol']=='School Holiday')+\
                            3*(df_std['School_Hol']=='Christmas Holiday')
    target = df[exog_name]
    return df_std[features], target

# GAM model for all regions
Target variable: Net-Load


In [17]:
from pygam import LinearGAM, s, f, l, te

target_variable = 'node'

X_train = {}
Y_train = {}

X_test = {}
Y_test = {}

for (reg, df) in zip(regions, regions_df):
    # Train set
    X_train[reg], Y_train[reg] = dataset_preprocessing(train_set[reg], features, features_to_standardize, target_variable)
    #Test set
    X_test[reg], Y_test[reg] = dataset_preprocessing(test_set[reg], features, features_to_standardize, target_variable)


# GAM_model = {}

# for reg in regions:
#     print(reg)
#     # gam_model = LinearGAM(l(0) + l(1) + l(2) + l(3) + l(4) + l(5) + l(6) + s(7,n_splines=35,basis='cp') + f(8) + f(9) + 
#     #                      te(7,8,n_splines=30,dtype=['numerical','categorical'],basis='cp')+te(7,9,n_splines=20,dtype=['numerical','categorical'],basis='cp')+
#     #                      s(10,n_splines=35,basis='cp')+s(11,n_splines=35,basis='cp')+s(12,n_splines=5,basis='cp')+ 
#     #                      te(13,14,n_splines=20,basis='cp')+l(15)+s(16,n_splines=5,basis='cp')+te(10,7,n_splines=17)+te(16,7,n_splines=17))
#     gam_model = LinearGAM(l(0) + l(1) + l(2) + l(3) + l(4) + l(5) + l(6) + s(7,n_splines=35,basis='cp') + f(8) + f(9) + 
#                          #te(7,8,n_splines=30,dtype=['numerical','categorical'],basis='cp')+te(7,9,n_splines=20,dtype=['numerical','categorical'],basis='cp')+
#                       s(7,by=8,n_splines=30,basis="cp") + s(7,by=9,n_splines=20,basis="cp")+    
#                       s(10,n_splines=35,basis='cp')+s(11,n_splines=35,basis='cp')+s(12,n_splines=5,basis='cp')+ 
#                       s(13,n_splines=20,by=14,basis="cp")+l(15)+s(16,n_splines=5,basis='cp')+
#                       #te(13,14,n_splines=20,basis='cp')
#                       te(10,7,n_splines=17)+te(16,7,n_splines=17))
    
#     gam_model.fit(np.array(X_train[reg]), np.array(Y_train[reg]))
#     GAM_model[reg] = gam_model

In [18]:
# with open('Saved_GAM/GAM_models.pkl', 'wb') as f:
#     pickle.dump(GAM_model, f)

In [19]:
with open('Saved_GAM/GAM_models.pkl', 'rb') as f:
    GAM_model = pickle.load(f)

In [20]:
y_GAM = {}

for reg in regions:
    y_GAM[reg] = GAM_model[reg].predict(np.array(X_test[reg]))
    
yhat_GAM_mat = np.zeros((len(test_set['A']),len(regions)))

for i,reg in enumerate(regions):
    yhat_GAM_mat[:,i] = y_GAM[reg]

### Evaluation of the results of GAM on all regions

In [21]:
idx_2020 = 17516
idx_2021 = 35104

print("\nnRMSE GAM on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_GAM_mat[:idx_2020]))
print("nRMSE GAM on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_GAM_mat[idx_2020:idx_2021]))
print("nRMSE GAM on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_GAM_mat[idx_2021:]))

print("\nnMAE GAM on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_GAM_mat[:idx_2020]))
print("nMAE GAM on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_GAM_mat[idx_2020:idx_2021]))
print("nMAE GAM on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_GAM_mat[idx_2021:]))


nRMSE GAM on 2019 test set:  0.7415244022214377
nRMSE GAM on 2020 test set:  0.7611470092471138
nRMSE GAM on 2021 test set:  0.907646160094514

nMAE GAM on 2019 test set:  0.6790128202891225
nMAE GAM on 2020 test set:  0.7209288635419329
nMAE GAM on 2021 test set:  0.8755985051323371


## Static Kalman Filter on GAM model

### 48 hours ahead prediction

In [22]:
idx_2020 = 17516
idx_2021 = 35104

kf_static = {}

for reg in regions:
    kf = Kalman_Filter(GAM_model[reg])
    kf.fit(X_train[reg], Y_train[reg])
    kf_static[reg] = kf

y_Kalman_static = {}

for reg in regions:
    # Prediction on test set
    df_kf = pd.DataFrame(test_set[reg][['targetTime', 'node']])
    y_Kalman_static[reg] = {}
    y_Kalman_static[reg]['2019'], _ = kf_static[reg].predict(X_test[reg][:idx_2020], df_kf.iloc[:idx_2020], delay=True) 
    y_Kalman_static[reg]['2020'], _ = kf_static[reg].predict(X_test[reg][idx_2020:idx_2021], df_kf.iloc[idx_2020:idx_2021], delay=True) 
    y_Kalman_static[reg]['2021'], _ = kf_static[reg].predict(X_test[reg][idx_2021:], df_kf.iloc[idx_2021:], delay=True) 

## Dynamic Kalman Filter on GAM model

### 48 hours ahead prediction

In [23]:
idx_2020 = 17516
idx_2021 = 35104

kf_dynamic = {}

for reg in regions:
    with open(f'DynamicKF_matrices_py/Q_{reg}.npy', 'rb') as f:
        Q = np.load(f)
        sigma = np.load(f)
    kf = Kalman_Filter(GAM_model[reg], Q=Q, sigma=sigma)
    kf.fit(X_train[reg], Y_train[reg])
    kf_dynamic[reg] = kf

y_Kalman_dynamic = {}

for reg in regions:
    # 48h prediction
    df_kf = pd.DataFrame(test_set[reg][['targetTime', 'node']])
    y_Kalman_dynamic[reg] = {}
    y_Kalman_dynamic[reg]['2019'], _ = kf_dynamic[reg].predict(X_test[reg][:idx_2020], df_kf.iloc[:idx_2020], delay=True) 
    y_Kalman_dynamic[reg]['2020'], _ = kf_dynamic[reg].predict(X_test[reg][idx_2020:idx_2021], df_kf.iloc[idx_2020:idx_2021], delay=True) 
    y_Kalman_dynamic[reg]['2021'], _ = kf_dynamic[reg].predict(X_test[reg][idx_2021:], df_kf.iloc[idx_2021:], delay=True) 

In [24]:
idx_2020 = 17516
idx_2021 = 35104
    
yhat_staticKF_mat = np.zeros((len(test_set['A']),len(regions)))
yhat_dynamicKF_mat = np.zeros((len(test_set['A']),len(regions)))

for i,reg in enumerate(regions):
    yhat_staticKF_mat[:idx_2020,i] = y_Kalman_static[reg]['2019']
    yhat_staticKF_mat[idx_2020:idx_2021,i] = y_Kalman_static[reg]['2020']
    yhat_staticKF_mat[idx_2021:,i] = y_Kalman_static[reg]['2021']
    yhat_dynamicKF_mat[:idx_2020,i] = y_Kalman_dynamic[reg]['2019']
    yhat_dynamicKF_mat[idx_2020:idx_2021,i] = y_Kalman_dynamic[reg]['2020']
    yhat_dynamicKF_mat[idx_2021:,i] = y_Kalman_dynamic[reg]['2021']


print("\nnRMSE GAM on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_GAM_mat[:idx_2020]))
print("nRMSE GAM on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_GAM_mat[idx_2020:idx_2021]))
print("nRMSE GAM on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_GAM_mat[idx_2021:]))

print("\nnMAE GAM on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_GAM_mat[:idx_2020]))
print("nMAE GAM on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_GAM_mat[idx_2020:idx_2021]))
print("nMAE GAM on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_GAM_mat[idx_2021:]))

print("\n\nnRMSE Static KF on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_staticKF_mat[:idx_2020]))
print("nRMSE Static KF on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_staticKF_mat[idx_2020:idx_2021]))
print("nRMSE Static KF on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_staticKF_mat[idx_2021:]))

print("\nnMAE Static KF on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_staticKF_mat[:idx_2020]))
print("nMAE Static KF on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_staticKF_mat[idx_2020:idx_2021]))
print("nMAE Static KF on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_staticKF_mat[idx_2021:]))

print("\n\nnRMSE Dynamic KF on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_dynamicKF_mat[:idx_2020]))
print("nRMSE Dynamic KF on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_dynamicKF_mat[idx_2020:idx_2021]))
print("nRMSE Dynamic KF on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_dynamicKF_mat[idx_2021:]))

print("\nnMAE Dynamic KF on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_dynamicKF_mat[:idx_2020]))
print("nMAE Dynamic KF on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_dynamicKF_mat[idx_2020:idx_2021]))
print("nMAE Dynamic KF on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_dynamicKF_mat[idx_2021:]))


nRMSE GAM on 2019 test set:  0.7415244022214377
nRMSE GAM on 2020 test set:  0.7611470092471138
nRMSE GAM on 2021 test set:  0.907646160094514

nMAE GAM on 2019 test set:  0.6790128202891225
nMAE GAM on 2020 test set:  0.7209288635419329
nMAE GAM on 2021 test set:  0.8755985051323371


nRMSE Static KF on 2019 test set:  0.4763459187359143
nRMSE Static KF on 2020 test set:  0.48765814554013326
nRMSE Static KF on 2021 test set:  0.4969960701935302

nMAE Static KF on 2019 test set:  0.4328284397771235
nMAE Static KF on 2020 test set:  0.4434304642778586
nMAE Static KF on 2021 test set:  0.4786110517463343


nRMSE Dynamic KF on 2019 test set:  0.47617096318586283
nRMSE Dynamic KF on 2020 test set:  0.49081850181607456
nRMSE Dynamic KF on 2021 test set:  0.4839336179059813

nMAE Dynamic KF on 2019 test set:  0.4295336037016229
nMAE Dynamic KF on 2020 test set:  0.43165588353723733
nMAE Dynamic KF on 2021 test set:  0.44349769084950086
